# VPR Data Landing

# 1. Import dependencies & declare constants

In [ ]:
import sys
import os
import yaml
from dotenv import load_dotenv, find_dotenv

# Add the src directory to the sys.path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

In [ ]:
from spark_session import create_spark_session
from schemas import *
from functions import *

In [ ]:
# Load the .env file from the parent directory
env_path = find_dotenv(filename=".env", raise_error_if_not_found=True)
load_dotenv(dotenv_path=env_path)

# Load AWS credentials
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")

In [ ]:
# Load the YAML configuration file
with open('../config/config.yml', 'r') as file:
    config = yaml.safe_load(file)

In [ ]:
BUCKET_NAME = config["paths"]["BUCKET_NAME"]
RAW = config["paths"]["RAW"]
ORDERS = config["paths"]["ORDERS"]

BRONZE = config["paths"]["BRONZE"]
SILVER = config["paths"]["SILVER"]
GOLD = config["paths"]["GOLD"]

ADDRESS_DATA = config["raw_data"]["ADDRESS_DATA"]
CLIENTS_DATA = config["raw_data"]["CLIENTS_DATA"]
PRODUCTS_DATA = config["raw_data"]["PRODUCTS_DATA"]

DATABASE_NAME = "vpr-optimizer-platforom_db"

ADDRESS_TABLE = config["table_names"]["ADDRESS_TABLE"]
CLIENTS_TABLE = config["table_names"]["CLIENTS_TABLE"]
CLIENTS_ADDRESS_TABLE = config["table_names"]["CLIENTS_ADDRESS_TABLE"]
PRODUCTS_TABLE = config["table_names"]["PRODUCTS_TABLE"]
PACKAGE_TABLE = config["table_names"]["PACKAGE_TABLE"]

RAW_ADDRESS_PATH = os.path.join(BUCKET_NAME, RAW, ADDRESS_DATA)
RAW_CIENTS_PATH = os.path.join(BUCKET_NAME, RAW, CLIENTS_DATA)
RAW_PRODUCTS_PATH = os.path.join(BUCKET_NAME, RAW, PRODUCTS_DATA)

BRONZE_ADDRESS_PATH = os.path.join(BUCKET_NAME, ORDERS, BRONZE, ADDRESS_TABLE)
BRONZE_CLIENTS_PATH = os.path.join(BUCKET_NAME, ORDERS, BRONZE, CLIENTS_TABLE)
BRONZE_PRODUCTS_PATH = os.path.join(BUCKET_NAME, ORDERS, BRONZE, PRODUCTS_TABLE)


SILVER_ADDRESS_PATH = os.path.join(BUCKET_NAME, ORDERS, SILVER, ADDRESS_TABLE)
SILVER_CLIENTS_PATH = os.path.join(BUCKET_NAME, ORDERS, SILVER, CLIENTS_TABLE)
SILVER_PRODUCTS_PATH = os.path.join(BUCKET_NAME, ORDERS, SILVER, PRODUCTS_TABLE)

GOLD_CLIENTS_ADDRESS_PATH = os.path.join(BUCKET_NAME, ORDERS, GOLD, CLIENTS_ADDRESS_TABLE)
GOLD_PRODUCTS_PATH = os.path.join(BUCKET_NAME, ORDERS, GOLD, PRODUCTS_TABLE)
GOLD_PACKAGE_PATH = os.path.join(BUCKET_NAME, ORDERS, GOLD, PACKAGE_TABLE)

# 2. Initialize Spark Session

In [ ]:
spark = create_spark_session(aws_access_key_id, aws_secret_access_key)

# 3. Medallion Architecure

## 3.1 Bronze Layer

In [ ]:
df_address_raw = read_file(spark, RAW_ADDRESS_PATH, "json", addresses_schema)
df_clients_raw = read_file(spark, RAW_CIENTS_PATH, "json", clients_schema)
df_products_raw = read_file(spark, RAW_PRODUCTS_PATH, "json", products_schema)

In [ ]:
write_df(df_address_raw, BRONZE_ADDRESS_PATH)
write_df(df_clients_raw, BRONZE_CLIENTS_PATH)
write_df(df_products_raw, BRONZE_PRODUCTS_PATH)

## 3.2 Silver Layer

In [ ]:
df_address_bronze = read_file(spark, BRONZE_ADDRESS_PATH, "parquet", addresses_schema)
df_clients_bronze = read_file(spark, BRONZE_CLIENTS_PATH, "parquet", clients_schema)
df_products_bronze = read_file(spark, BRONZE_PRODUCTS_PATH, "parquet", products_schema)

In [ ]:
write_df(transform_addresses_bronze_to_silver(df_address_bronze), SILVER_ADDRESS_PATH)
write_df(transform_clients_bronze_to_silver(df_clients_bronze), SILVER_CLIENTS_PATH)
write_df(transform_products_bronze_to_silver(df_products_bronze), SILVER_PRODUCTS_PATH)

## 3.3 Gold Layer

In [ ]:
df_address_silver = read_file(spark, SILVER_ADDRESS_PATH, "parquet", silver_address_schema)
df_clients_silver = read_file(spark, SILVER_CLIENTS_PATH, "parquet", silver_clients_schema)
df_products_silver = read_file(spark, SILVER_PRODUCTS_PATH, "parquet", silver_products_schema)

In [ ]:
write_df(transform_clients_addresses_silver_to_gold(df_clients_silver, df_address_silver), GOLD_CLIENTS_ADDRESS_PATH)
write_df(transform_products_silver_to_gold(df_products_silver), GOLD_PRODUCTS_PATH, file_type="delta")
write_df(transform_packages_silver_to_gold(df_products_silver), GOLD_PACKAGE_PATH, file_type="delta")